In [1]:
import pandas as pd
from datetime import datetime
import numpy as np 
pd.set_option('display.max_columns', None)
from datetime import timedelta
import sqlite3


In [2]:
conn = sqlite3.connect("/home/sam/Documents/DSI/capstone/poker/data/dh_backup_20210330_081745/drivehud.db")
hand_history = pd.read_sql_query("select * from HandHistories;", conn)
hand_history.HandHistory = hand_history.HandHistory.str.split('\r\n')

df = hand_history.copy()
df_full = pd.read_csv('src/datacleaning/playing_around.csv')



In [79]:
def vpip(df):
    '''
    finds vpip of player prior to the hand currently being played 
    '''
    #df['prior_actions'] = None 
    total_dic = {} # probably replace this w/ assignment directly to the dataframe 
    unique_TN = df['TournamentNumber'].unique()
    for num in unique_TN: # iterates through all unique tournament numbers
        mask = df['TournamentNumber'] == num
        tourn_df = df[mask]
        for idx, row in tourn_df.iterrows(): # go through each row of tournament 
            row_dic = {}
            mask = tourn_df.index < idx
            df_prior = tourn_df[mask] # hands that occured previously in this tournament 
            for player in row['player_names']: # looping through all players present in hand
                player_actions = []
                for dic in df_prior['action_type']: # looking at all prior hands 
                    if player in dic:
                        player_actions.append(dic[player])
                row_dic[player] = player_actions
            total_dic[idx] = row_dic
    return total_dic

In [80]:
vpip(df)

{0: {'P18_572249IY': [],
  'P14_459713MR': [],
  'P25_871309MO': [],
  'P3_286963JX': [],
  'P5_405793KB': [],
  'Hero': [],
  'P26_251842VD': [],
  'P4_131206CX': [],
  'P11_924370QJ': []},
 1: {'P18_572249IY': [0],
  'P14_459713MR': [0],
  'P25_871309MO': [23],
  'P3_286963JX': [0],
  'P5_405793KB': [0],
  'Hero': [0],
  'P26_251842VD': [3],
  'P4_131206CX': [0],
  'P11_924370QJ': [0]},
 2: {'P18_572249IY': [0, 0],
  'P14_459713MR': [0, 3],
  'P25_871309MO': [23, 3],
  'P3_286963JX': [0, 0],
  'P5_405793KB': [0, 0],
  'Hero': [0, 23],
  'P26_251842VD': [3, 0],
  'P4_131206CX': [0, 0],
  'P11_924370QJ': [0, 0]},
 3: {'P18_572249IY': [0, 0, 0],
  'P25_871309MO': [23, 3, 0],
  'P3_286963JX': [0, 0, 0],
  'P5_405793KB': [0, 0, 3],
  'Hero': [0, 23, 0],
  'P26_251842VD': [3, 0, 0],
  'P4_131206CX': [0, 0, 0],
  'P11_924370QJ': [0, 0, 0]},
 4: {'P18_572249IY': [0, 0, 0, 0],
  'P25_871309MO': [23, 3, 0, 0],
  'P3_286963JX': [0, 0, 0, 0],
  'P5_405793KB': [0, 0, 3, 0],
  'Hero': [0, 23, 0, 0

In [70]:
df['player_names'] = df['HandHistory'].apply(lambda x: player_names(x))

In [69]:
df['action_type'] = df.HandHistory.apply(lambda x: action_type(x))


In [75]:
mask = df.TournamentNumber != ''
df = df[mask]

In [76]:
df

,HandHistoryId,HandNumber,PokerSiteId,HandHistory,HandHistoryTimestamp,GameType,TournamentNumber,action_type,player_names
0,1,3951210633,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-24 03:08:08,1,459742,"{'P26_251842VD': 3, 'P4_131206CX': 0, 'P11_924...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_..."
1,2,3951211988,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-24 03:09:34,1,459742,"{'P4_131206CX': 0, 'P11_924370QJ': 0, 'P18_572...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_..."
2,3,3951212830,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-24 03:11:47,1,459742,"{'P11_924370QJ': 0, 'P18_572249IY': 0, 'P14_45...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_..."
3,4,3951214131,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-24 03:13:37,1,459742,"{'P18_572249IY': 0, 'P25_871309MO': 0, 'P3_286...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4..."
4,5,3951215154,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-24 03:14:31,1,459742,"{'P25_871309MO': 0, 'P3_286963JX': 0, 'P5_4057...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4..."
...,...,...,...,...,...,...,...,...,...
54299,54333,4286799273,1,"[<session sessioncode=""3408296"">, <general>, <...",2021-03-30 14:12:25,82,41343764,"{'P6_773472PB': 3, 'Hero': 7}","[P6_773472PB, Hero]"
54300,54334,4286799339,1,"[<session sessioncode=""4917292"">, <general>, <...",2021-03-30 14:12:29,82,41343587,{'Hero': 0},"[P1_574120EG, Hero]"
54301,54335,4286799383,1,"[<session sessioncode=""3408296"">, <general>, <...",2021-03-30 14:12:34,82,41343764,{'Hero': 0},"[P6_773472PB, Hero]"
54302,54336,4286799396,1,"[<session sessioncode=""4917292"">, <general>, <...",2021-03-30 14:12:41,112,41343587,{},"[P1_574120EG, Hero]"


In [18]:
df.HandHistory.iloc[40006]

['<session sessioncode="1968702">',
 '<general>',
 '<client_version>19.9.7.4</client_version>',
 '<uncalled_bet_enabled>false</uncalled_bet_enabled>',
 '<mode>real</mode>',
 '<gametype>Holdem NL</gametype>',
 '<tablename>$400 Guaranteed (Super Turbo 10K Chips) (#38261159)</tablename>',
 '<duration>N/A</duration>',
 '<gamecount>0</gamecount>',
 '<startdate>2020-12-24 06:15:11</startdate>',
 '<currency>USD</currency>',
 '<nickname>Hero</nickname>',
 '<bets>0</bets>',
 '<wins>0</wins>',
 '<chipsin>0</chipsin>',
 '<chipsout>0</chipsout>',
 '<ipoints>0</ipoints>',
 '<statuspoints>0</statuspoints>',
 '<awardpoints>0</awardpoints>',
 '<tournamentcode>38261159</tournamentcode>',
 '<tournamentname>$400 Guaranteed (Super Turbo 10K Chips) (#38261159)</tournamentname>',
 '<place>0</place>',
 '<buyin>$3+$0.30</buyin>',
 '<totalbuyin>$3.30</totalbuyin>',
 '<win>0</win>',
 '<maxplayers>9</maxplayers>',
 '<pokersite>Ignition</pokersite>',
 '</general>',
 '<game gamecode="4222046472">',
 '<general>',
 

In [74]:
df.TournamentNumber.iloc[1000]

''

In [24]:
df.HandHistory.iloc[40002]

['<session sessioncode="1968702">',
 '<general>',
 '<client_version>19.9.7.4</client_version>',
 '<uncalled_bet_enabled>false</uncalled_bet_enabled>',
 '<mode>real</mode>',
 '<gametype>Holdem NL</gametype>',
 '<tablename>$400 Guaranteed (Super Turbo 10K Chips) (#38261159)</tablename>',
 '<duration>N/A</duration>',
 '<gamecount>0</gamecount>',
 '<startdate>2020-12-24 06:15:11</startdate>',
 '<currency>USD</currency>',
 '<nickname>Hero</nickname>',
 '<bets>0</bets>',
 '<wins>0</wins>',
 '<chipsin>0</chipsin>',
 '<chipsout>0</chipsout>',
 '<ipoints>0</ipoints>',
 '<statuspoints>0</statuspoints>',
 '<awardpoints>0</awardpoints>',
 '<tournamentcode>38261159</tournamentcode>',
 '<tournamentname>$400 Guaranteed (Super Turbo 10K Chips) (#38261159)</tournamentname>',
 '<place>0</place>',
 '<buyin>$3+$0.30</buyin>',
 '<totalbuyin>$3.30</totalbuyin>',
 '<win>0</win>',
 '<maxplayers>9</maxplayers>',
 '<pokersite>Ignition</pokersite>',
 '</general>',
 '<game gamecode="4222043627">',
 '<general>',
 

In [21]:
df.HandHistory.iloc[40008]

['<session sessioncode="1968702">',
 '<general>',
 '<client_version>19.9.7.4</client_version>',
 '<uncalled_bet_enabled>false</uncalled_bet_enabled>',
 '<mode>real</mode>',
 '<gametype>Holdem NL</gametype>',
 '<tablename>$400 Guaranteed (Super Turbo 10K Chips) (#38261159)</tablename>',
 '<duration>N/A</duration>',
 '<gamecount>0</gamecount>',
 '<startdate>2020-12-24 06:15:11</startdate>',
 '<currency>USD</currency>',
 '<nickname>Hero</nickname>',
 '<bets>0</bets>',
 '<wins>0</wins>',
 '<chipsin>0</chipsin>',
 '<chipsout>0</chipsout>',
 '<ipoints>0</ipoints>',
 '<statuspoints>0</statuspoints>',
 '<awardpoints>0</awardpoints>',
 '<tournamentcode>38261159</tournamentcode>',
 '<tournamentname>$400 Guaranteed (Super Turbo 10K Chips) (#38261159)</tournamentname>',
 '<place>0</place>',
 '<buyin>$3+$0.30</buyin>',
 '<totalbuyin>$3.30</totalbuyin>',
 '<win>0</win>',
 '<maxplayers>9</maxplayers>',
 '<pokersite>Ignition</pokersite>',
 '</general>',
 '<game gamecode="4222048018">',
 '<general>',
 

In [23]:
df.HandHistory.iloc[40009]

['<session sessioncode="1968702">',
 '<general>',
 '<client_version>19.9.7.4</client_version>',
 '<uncalled_bet_enabled>false</uncalled_bet_enabled>',
 '<mode>real</mode>',
 '<gametype>Holdem NL</gametype>',
 '<tablename>$400 Guaranteed (Super Turbo 10K Chips) (#38261159)</tablename>',
 '<duration>N/A</duration>',
 '<gamecount>0</gamecount>',
 '<startdate>2020-12-24 06:15:11</startdate>',
 '<currency>USD</currency>',
 '<nickname>Hero</nickname>',
 '<bets>0</bets>',
 '<wins>0</wins>',
 '<chipsin>0</chipsin>',
 '<chipsout>0</chipsout>',
 '<ipoints>0</ipoints>',
 '<statuspoints>0</statuspoints>',
 '<awardpoints>0</awardpoints>',
 '<tournamentcode>38261159</tournamentcode>',
 '<tournamentname>$400 Guaranteed (Super Turbo 10K Chips) (#38261159)</tournamentname>',
 '<place>0</place>',
 '<buyin>$3+$0.30</buyin>',
 '<totalbuyin>$3.30</totalbuyin>',
 '<win>0</win>',
 '<maxplayers>9</maxplayers>',
 '<pokersite>Ignition</pokersite>',
 '</general>',
 '<game gamecode="4222048797">',
 '<general>',
 

In [36]:
df.HandHistory.iloc[85]

['<session sessioncode="459742">',
 '<general>',
 '<mode>real</mode>',
 '<gametype>Holdem NL</gametype>',
 '<tablename>3-Table Sit &amp; Go (#30839441)</tablename>',
 '<duration>N/A</duration>',
 '<gamecount>0</gamecount>',
 '<startdate>2019-10-24 03:06:00</startdate>',
 '<currency>USD</currency>',
 '<nickname>Hero</nickname>',
 '<bets>0</bets>',
 '<wins>0</wins>',
 '<chipsin>0</chipsin>',
 '<chipsout>0</chipsout>',
 '<ipoints>0</ipoints>',
 '<statuspoints>0</statuspoints>',
 '<awardpoints>0</awardpoints>',
 '<tournamentname>3-Table Sit &amp; Go (#30839441)</tournamentname>',
 '<place>0</place>',
 '<buyin>$1+$0.10</buyin>',
 '<totalbuyin>$1.10</totalbuyin>',
 '<win>0</win>',
 '<maxplayers>9</maxplayers>',
 '<pokersite>Ignition</pokersite>',
 '</general>',
 '<game gamecode="3951255714">',
 '<general>',
 '<startdate>2019-10-24 04:31:40</startdate>',
 '<players>',
 '<player seat="6" name="P22_272545FH" chips="2690" dealer="0" win="3640" bet="2690" rebuy="0" addon="0" />',
 '<player seat="

In [5]:
train = pd.read_csv('data/train_classification_tournaments.csv') # 8556 ---total:10695

In [4]:
hold = pd.read_csv('data/updated_DF.csv') #2139

In [7]:
mask = hold.river_bet >0
hold[mask]
hold.HandHistory.iloc[54121].split()

'[\'<session sessioncode="3607122">\', \'<general>\', \'<client_version>19.9.7.4</client_version>\', \'<uncalled_bet_enabled>false</uncalled_bet_enabled>\', \'<mode>real</mode>\', \'<gametype>Holdem NL</gametype>\', \'<tablename>Hyper Turbo (500 Chips) (#41304105)</tablename>\', \'<duration>N/A</duration>\', \'<gamecount>0</gamecount>\', \'<startdate>2021-03-29 15:40:43</startdate>\', \'<currency>USD</currency>\', \'<nickname>Hero</nickname>\', \'<bets>0</bets>\', \'<wins>0</wins>\', \'<chipsin>0</chipsin>\', \'<chipsout>0</chipsout>\', \'<ipoints>0</ipoints>\', \'<statuspoints>0</statuspoints>\', \'<awardpoints>0</awardpoints>\', \'<tournamentcode>41304105</tournamentcode>\', \'<tournamentname>Hyper Turbo (500 Chips) (#41304105)</tournamentname>\', \'<place>6</place>\', \'<buyin>$1+$0.05</buyin>\', \'<totalbuyin>$1.05</totalbuyin>\', \'<win>0</win>\', \'<maxplayers>6</maxplayers>\', \'<pokersite>Ignition</pokersite>\', \'</general>\', \'<game gamecode="4286208914">\', \'<general>\', \

In [ ]:
df['hour'] = df['HandHistoryTimestamp'].apply(lambda x: x.hour) # will need to determine degree to which time is offset
df['year_month_day'] = pd.to_datetime(df['HandHistoryTimestamp']).dt.to_period('D').dt.to_timestamp()  # will need to determine degree to which time is offset
df['days_since_start'] = df['year_month_day'].apply(lambda x: (x - pd.to_datetime('2019-10-23 00:00:00')).days)


In [ ]:
df[['HandHistoryTimestamp', 'hour', 'year_month_day', 'days_since_start']]

In [2]:
df = pd.read_csv('data/updated_DF.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54304 entries, 0 to 54303
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 54304 non-null  int64  
 1   HandHistoryId              54304 non-null  int64  
 2   HandNumber                 54304 non-null  int64  
 3   PokerSiteId                54304 non-null  int64  
 4   HandHistory                54304 non-null  object 
 5   HandHistoryTimestamp       54304 non-null  object 
 6   GameType                   54304 non-null  int64  
 7   TournamentNumber           39551 non-null  float64
 8   buyin                      54304 non-null  float64
 9   cash_or_tourn              54304 non-null  object 
 10  my_blind_anti_total        54304 non-null  float64
 11  my_cards                   53293 non-null  object 
 12  starting_stack             53293 non-null  float64
 13  tournament_type            39551 non-null  obj

In [ ]:
def vpip(df):
    '''
    finds vpip of player prior to the hand currently being played 
    '''
    unique_TN = df['TournamentNumber'].unique()
    for num in unique_TN: # iterates through all unique tournament numbers
        mask = df['TournamentNumber'] = num
        tourn_df = df[mask]
        for index, row in tourn_df.iterrows():
            

In [ ]:
def action_type(x):
    '''
    finds all action types a player made preflop 
    '''
    result = {}
    start = 0
    stop = 0
    player = None
    action_type = None 
    for idx, elem in enumerate(x): # finds start of preflop action 
        if 'Pocket' in elem:
            start = idx
    subset = x[start+1:]
    for idx, elem in enumerate(subset):# finds end of preflop action
        if '</round>' in elem:
            stop = idx
            break 
    subset = subset[:stop]
    
    for elem in subset: # iterating through preflop action to find player/type 
        for sub_elem in elem.split():
            if 'player' in sub_elem:
                player = sub_elem.replace('player="', '').replace('"', '')
            if 'type' in sub_elem:
                action_type = int(sub_elem.replace('type="', '').replace('"', ''))
        if player in result: # checks to see if player has already taken an action 
            break
        else:
            result[player] = action_type
    return result 

In [ ]:
def preflop_actions(x):
    '''
    creates a dictionary of whether each player raised or bet each hand 
    we'll let a 2 = PFR
    and a 1 = VPIP 
    '''
    BB = x['BB']
    HH = X['HandHistory']
    for elem in BB:
        if B

In [ ]:
# rather than doing this, lets create a feature for each row that has a dictionary w/ the actions of each player,
# then use that to create what we're trying to do here 
def preflop_stats(df):
    unique_TN = df['TournamentNumber'].unique()
    for num in unique_TN: # iterates through all unique tournament numbers
        mask = df['TournamentNumber'] = num
        tourn_df = df[mask]
        for index, row in tourn_df.iterrows():